In [1]:
IRT_DIMS = 0
DATASET = 'matrix'    
ITEM_FEATURES = 'False'
ITEM_FEATURES = False if ITEM_FEATURES == 'False' else True

In [2]:
import sys, os, shutil
import pickle
import time
import numpy as np
from utils import (mymode, load_data_heterogeneous, create_data_object_heterogeneous)
import seaborn as sns

from IRT import MIRT_2PL
from Heterogeneous_embedder import EmbedderHeterogeneous, train_embedder_heterogeneous, test_embedder_heterogeneous
from manage_experiments import perform_cross_validation

import torch
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import pearsonr

OSError: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory

In [ ]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# Initialise
parameters = {
    'df_name': None,
    'epochs': 10000,
    'learning_rate': 0.005,
    'weight_decay': 0,
    'early_stopping': 200,
    'n_splits': 10,
    'device': 'cuda:0',
    'batch_size': 2**14,
    'neighbours': [50, 50]
    }

if IRT_DIMS > 0:
    parameters['model_type'] = 'IRT'
    parameters['hidden_dims'] = IRT_DIMS
    parameters['lambda1'] = 0
    parameters['lambda2'] = 0
    OUTNAME = 'IRT'
else:
    parameters['model_type'] = 'GNN'
    parameters['hidden_dims'] = [16,8]
    OUTNAME = 'SAGE' 

if ITEM_FEATURES:
    OUTNAME = OUTNAME + '_scales'
    
print(parameters)
print(DATASET)

In [5]:
DATA_FILE = 'data/mindsteps_set_' + DATASET
df = load_data_heterogeneous(DATA_FILE)
data, df_student, df_item, df_edge = create_data_object_heterogeneous(df, return_aux_data=True, item_features=ITEM_FEATURES)

In [6]:
df.columns

Index(['studentId', 'code', 'motherTongue', 'Gender', 'scale', 'matrix',
       'IRT_difficulty', 'topic', 'responseformat', 'textlength', 'score',
       'age', 'grade', 'viewingTime', 'timestamp', 'useCase', 'assessmentId',
       'ability', 'frequency', 'previous_sessions', 'years_from_start',
       'matdiff', 'matcode', 'domain'],
      dtype='object')

In [7]:
df.head()

,studentId,code,motherTongue,Gender,scale,matrix,IRT_difficulty,topic,responseformat,textlength,...,timestamp,useCase,assessmentId,ability,frequency,previous_sessions,years_from_start,matdiff,matcode,domain
0,0,0,1.0,0.0,fhoe,F.1.A.1.c.1,-0.370330,Monologe und Dialoge,NaN,NaN,...,7.830814e-312,ms-kber,85007E334A2C9F19ECA408647E6F97139649B1465E15EB...,1.083005,1.851849,2,1.08,,,f
1,0,1,1.0,0.0,fles,F.2.A.1.b.3,-0.494880,Berichte,NaN,NaN,...,7.833807e-312,ms-thema-auto,0D59C4843FFC39195EDE1DD140506CF3DF8BC6F132AAF5...,0.657010,2.727266,3,1.10,,,f
2,0,2,1.0,0.0,mzuv,MA.1.A.3.f,0.368830,Gemischte Grundoperationen,single choice,NaN,...,7.661659e-312,ms-kber,2B0D3CD783DB6BE2341F8CF101395AC464874A993E8CCF...,0.751265,1000000.000000,0,0.00,,,m
3,0,3,1.0,0.0,fhoe,F.1.A.1.d.1,0.367442,Aus TV und Radio,NaN,NaN,...,7.830815e-312,ms-kber,85007E334A2C9F19ECA408647E6F97139649B1465E15EB...,1.083005,1.851849,2,1.08,,,f
4,0,4,1.0,0.0,fles,F.2.A.1.d.1,1.160150,Berichte,NaN,212.0,...,7.833807e-312,ms-thema-auto,0D59C4843FFC39195EDE1DD140506CF3DF8BC6F132AAF5...,0.657010,2.727266,3,1.10,,,f


edge_dim = data['student', 'responds', 'item'].edge_attr.shape[1]
if IRT_DIMS > 0:
    model = MIRT_2PL(IRT_DIMS, edge_dim, data)
else:
    model = EmbedderHeterogeneous( 
        n_students =  data['student'].x.size(0),
        n_items = data['item'].x.size(0),
        student_inchannel = data['student'].x.size(1),
        item_inchannel = data['item'].x.size(1),
        hidden_channels=parameters['hidden_dims'],
        edge_channel=edge_dim,
        metadata=data.metadata()
        ).to(device)
    
model = model.to(device)

In [ ]:
output_dict, model = perform_cross_validation(data, parameters, save_embeddings=True, save_subgraph=True, final_fit=True)

0it [00:00, ?it/s]

Parameters
{'n_students': 51508, 'n_items': 17868, 'student_inchannel': 2, 'item_inchannel': None, 'hidden_channels': [16, 8], 'edge_channel': 2}



  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:12<00:36, 12.05s/it]

 50%|█████     | 2/4 [00:16<00:14,  7.45s/it]

 75%|███████▌  | 3/4 [00:20<00:05,  5.92s/it]

100%|██████████| 4/4 [00:23<00:00,  5.84s/it]



Epoch: 001, Loss: 0.5498, Val: 0.5000, Test: 0.5000



  0%|          | 1/10000 [00:24<67:42:14, 24.38s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:10,  3.44s/it]

 50%|█████     | 2/4 [00:07<00:07,  3.52s/it]

 75%|███████▌  | 3/4 [00:10<00:03,  3.52s/it]

100%|██████████| 4/4 [00:13<00:00,  3.37s/it]


...


Epoch: 071, Loss: 0.4007, Val: 0.6277, Test: 0.6237



  1%|          | 71/10000 [17:08<41:41:27, 15.12s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:11,  3.87s/it]

 50%|█████     | 2/4 [00:07<00:07,  3.78s/it]

 75%|███████▌  | 3/4 [00:10<00:03,  3.57s/it]

100%|██████████| 4/4 [00:13<00:00,  3.46s/it]

  1%|          | 72/10000 [17:22<41:04:57, 14.90s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:10,  3.59s/it]

 50%|█████     | 2/4 [00:07<00:07,  3.60s/it]

 75%|███████▌  | 3/4 [00:10<00:03,  3.66s/it]

100%|██████████| 4/4 [00:14<00:00,  3.62s/it]

  1%|          | 73/10000 [17:37<41:10:19, 14.93s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:10,  3.60s/it]

 50%|█████     | 2/4 [00:07<00:07,  3.65s/it]

 75%|███████▌  | 3/4 [00:11<00:03,  3.70s/it]

100%|██████████| 4/4 [00:14<00:00,  3.53s/it]

  1%|          | 74/10000 [17:52<40:56:52, 14.85s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:11,  3.81s/it]

 50%|█████  


Epoch: 107, Loss: 0.4004, Val: 0.6280, Test: 0.6245



  1%|          | 107/10000 [25:41<39:51:51, 14.51s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:10,  3.58s/it]

 50%|█████     | 2/4 [00:07<00:07,  3.51s/it]

 75%|███████▌  | 3/4 [00:10<00:03,  3.33s/it]

100%|██████████| 4/4 [00:13<00:00,  3.26s/it]

  1%|          | 108/10000 [25:55<39:02:50, 14.21s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:10,  3.43s/it]

 50%|█████     | 2/4 [00:06<00:06,  3.38s/it]

 75%|███████▌  | 3/4 [00:09<00:03,  3.25s/it]

100%|██████████| 4/4 [00:12<00:00,  3.18s/it]

  1%|          | 109/10000 [26:08<38:11:54, 13.90s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:10,  3.39s/it]

 50%|█████     | 2/4 [00:06<00:06,  3.29s/it]

 75%|███████▌  | 3/4 [00:09<00:03,  3.24s/it]

100%|██████████| 4/4 [00:12<00:00,  3.19s/it]

  1%|          | 110/10000 [26:21<37:37:43, 13.70s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:10,  3.53s/it]

 50%|███

 75%|███████▌  | 3/4 [00:10<00:03,  3.50s/it]

100%|██████████| 4/4 [00:13<00:00,  3.33s/it]

  2%|▏         | 159/10000 [37:58<37:42:26, 13.79s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:09,  3.26s/it]

 50%|█████     | 2/4 [00:06<00:06,  3.29s/it]

 75%|███████▌  | 3/4 [00:09<00:03,  3.25s/it]

100%|██████████| 4/4 [00:12<00:00,  3.23s/it]

  2%|▏         | 160/10000 [38:12<37:21:32, 13.67s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:09,  3.29s/it]

 50%|█████     | 2/4 [00:06<00:06,  3.25s/it]

 75%|███████▌  | 3/4 [00:09<00:03,  3.26s/it]

100%|██████████| 4/4 [00:12<00:00,  3.17s/it]

  2%|▏         | 161/10000 [38:25<36:57:16, 13.52s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:11,  3.91s/it]

 50%|█████     | 2/4 [00:07<00:07,  3.55s/it]

 75%|███████▌  | 3/4 [00:10<00:03,  3.56s/it]

100%|██████████| 4/4 [00:13<00:00,  3.43s/it]

  2%|▏         | 162/10000 [38:39<37:32:00, 13.73s/it]

 

In [ ]:
output_dict.keys()
print('AUC:', output_dict['AUC_0_test'])
print('Balanced Accuracy:', output_dict['Balanced Accuracy_0_test'])

In [ ]:
unique_scales = df_item['scale'].unique()
unique_domains = df_item['domain'].unique()
unique_matdiff = df_item['matdiff'].sort_values().unique()
#scale_colors = dict([(c, plt.cm.tab10(i)) for i, c in enumerate(unique_scales)])
#domain_colors = dict([(c, plt.cm.tab10(i)) for i, c in enumerate(unique_domains)])

In [ ]:
train_losses = output_dict['losses_0']
train_edge_indices, val_edge_indices, test_edge_indices = output_dict['indices_0']

train_data = output_dict['train_subgraph_data'] 
val_data = output_dict['val_subgraph_data'] 
test_data = output_dict['test_subgraph_data']

aux_data = (df, df_student, df_item, df_edge, 
    #clustering_indices, 
    train_losses, #test_losses, test_aucs, 
    train_edge_indices, val_edge_indices, test_edge_indices, 
    data.cpu(), train_data.cpu(), val_data.cpu(), test_data.cpu(),
    unique_scales, unique_domains, unique_matdiff, 
    DATA_FILE)

with open(f'./results/{OUTNAME}_{DATASET}_aux_data.pkl', 'wb') as handle:
    pickle.dump(aux_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# SAVE EVERYTHING
torch.save(model, f'./results/{OUTNAME}_{DATASET}.pth.tar')

In [ ]:
train_indices = np.arange(len(train_losses))
fig = plt.figure()
plt.plot(train_indices, train_losses, c='blue')
plt.ylabel('Loss')
plt.xlabel('Epoch')

fig.tight_layout()

In [ ]:
# save IRT parameters to matrix
if OUTNAME == 'IRT' and IRT_DIMS == 1:    
    #z_dict = model.get_embeddings(train_data.to(device))
    z_dict = output_dict['embedding_0']
    df_item['IRT1_difficulty'] = -z_dict['offset']
    df_item['IRT1_discrimination'] = z_dict['discrimination']
    df_item['IRT1_discrimination_transf'] = z_dict['item']
    ability = z_dict['ability']
    df_edge['IRT1_ability'] = ability.ravel()
    
    aux_data = (df, df_student, df_item, df_edge, 
    train_losses, 
    train_edge_indices, val_edge_indices, test_edge_indices,
    data.cpu(), train_data.cpu(), val_data.cpu().cpu(), test_data.cpu(),
    unique_scales, unique_domains, unique_matdiff, 
    DATA_FILE)

    with open(f'./results/{OUTNAME}_{DATASET}_aux_data_IRT1.pkl', 'wb') as handle:
        pickle.dump(aux_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    df_item_clean = df_item.dropna(subset=['IRT_difficulty', 'IRT1_difficulty'])    
    
    fig = plt.figure()
    sns.scatterplot(x='IRT_difficulty', y='IRT1_difficulty', data=df_item, hue='scale')
    plt.title('Difficulty')
    print('Difficulty:', pearsonr(df_item_clean['IRT1_difficulty'], df_item_clean['IRT_difficulty']))
    
    edge_feat = train_data['student', 'responds', 'item'].edge_attr.detach().cpu().numpy()
    df_edge_clean = df_edge.dropna(subset=['IRT1_ability', 'ability', 'age'])
    fig = plt.figure()
    sns.scatterplot(x='age', y='IRT1_ability', data=df_edge_clean, hue='grade')
    plt.title('Age-Ability')
    
    fig = plt.figure()
    sns.scatterplot(x='grade', y='IRT1_ability', data=df_edge_clean, hue='age')
    plt.title('Grade-Ability')
    print('Age-Ability:', pearsonr(df_edge_clean['age'], df_edge_clean['IRT1_ability']))

    fig = plt.figure()
    sns.scatterplot(x='ability', y='IRT1_ability', data=df_edge_clean, hue='grade')
    plt.title('Ability')
    print('Ability:', pearsonr(df_edge_clean['ability'], df_edge_clean['IRT1_ability']))
    